# **Installing packages and importing libraries**

In [ ]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

!apt-get install -y -qq glpk-utils

!apt-get install -y -qq coinor-cbc



# **Ex2 : (Part 1-4) MILP Model and solver **



Solving the following MILP problem with the coefficients given in the text file.



 \begin{array}{l}
max\ \ \ \ \ \sum ^{n}_{j=1} c_{j} x_{j}\\
\\
s.t\ \ \sum ^{n}_{j=1} a_{ij} x_{j} \ \leqslant \ b_{i} \ ,     \ i\ =\ 1,2,......,m\\
x_{j\ } \in \ \{0,1\} ,\ \ \ \ j=1,2,.....,n
\end{array}




In [ ]:
# Creating model
model = ConcreteModel()

#importing the txt file of coefficients
coeff = np.loadtxt('lab4_ex2_lp_ip_coef.txt', delimiter=',')


# Numbers of variable N, and number of constraints M
N = coeff.shape[1] - 1
M = coeff.shape[0] -1


col_index = np.arange(N)
row_index = np.arange(M)

#Coefficients of constraints matrix with rhs
coeff_cons_with_rhs = coeff[1:,:]



# creating variables and their type as binary
model.x = Var(col_index, domain=Binary)


# Initialising list of constraints
model.cons = ConstraintList()

# Model objective
model.obj = Objective(expr = summation(coeff[0,:-1],model.x), sense = maximize)

#Adding constraits 

for i in row_index:
  model.cons.add(summation(coeff_cons_with_rhs[i,:-1],model.x) <= coeff_cons_with_rhs[i,-1])



# Solving the model with cbc solver
cbc_solver = SolverFactory('cbc')
result = cbc_solver.solve(model)


# Printing result

print('Status : ', result.solver.Status)
print('Terminating condition : ', result.solver.termination_condition)
print('Message : ', result.solver.termination_message)

print('\n')

print('The optimal value of the objective function is : ' , model.obj())
print('Time taken by the solver : ', result.solver.time)
print('\n')

print('Statistics : ' )
print('\n')
print(result.solver.Statistics)



Status :  ok
Terminating condition :  optimal
Message :  Model was solved to optimality (subject to tolerances), and an optimal solution is available.


The optimal value of the objective function is :  295828.0
Time taken by the solver :  69.12982439994812


Statistics : 



Branch and bound: 
  Number of bounded subproblems: 347044
  Number of created subproblems: 347044
Black box: 
  Number of iterations: 1407089



# Ex.2 Part 5-6 , LP Model and solver


 \begin{array}{l}
max\ \ \ \ \ \sum ^{n}_{j=1} c_{j} x_{j}\\
\\
s.t\ \ \sum ^{n}_{j=1} a_{ij} x_{j} \ \leqslant \ b_{i} \ ,     \ i\ =\ 1,2,......,m\\
0 \ \leqslant \ x_{j} \ \leqslant \ 1,\ \ \ \ j=1,2,.....,n
\end{array}


In [ ]:
# Creating model
model2 = ConcreteModel()

#importing the txt file of coefficients
coeff = np.loadtxt('lab4_ex2_lp_ip_coef.txt', delimiter=',')


# Numbers of variable N, and number of constraints M
N = coeff.shape[1] - 1
M = coeff.shape[0] -1


col_index = np.arange(N)
row_index = np.arange(M)

#Coefficients of constraints matrix with rhs
coeff_cons_with_rhs = coeff[1:,:]



# creating variables and their type as binary
model2.x = Var(col_index)


# Initialising list of constraints
model2.cons = ConstraintList()

# Model objective
model2.obj = Objective(expr = summation(coeff[0,:-1],model2.x), sense = maximize)

#Adding constraits 

for i in row_index:
  model2.cons.add(summation(coeff_cons_with_rhs[i,:-1],model2.x) <= coeff_cons_with_rhs[i,-1])

for i in col_index:
  model2.x[i].setub(1)
  model2.x[i].setlb(0)



# Solving the model with cbc solver
cbc_solver = SolverFactory('cbc')
result2 = cbc_solver.solve(model2)


# Printing result

print('Status : ', result2.solver.Status)
print('Terminating condition : ', result2.solver.termination_condition)
print('Message : ', result2.solver.termination_message)

print('\n')

print('The optimal value of the objective function is : ' , model2.obj())
print('Time taken by the solver : ', result2.solver.time)
print('\n')

print('Statistics : ' )
print('\n')
print(result2.solver.Statistics)



Status :  ok
Terminating condition :  optimal
Message :  Model was solved to optimality (subject to tolerances), and an optimal solution is available.


The optimal value of the objective function is :  295896.37697236
Time taken by the solver :  0.034960031509399414


Statistics : 



Branch and bound: 
  Number of bounded subproblems: None
  Number of created subproblems: None
Black box: 
  Number of iterations: 18



# Remarks : Comparison between models

So we observe that the time taken by the LP relaxtion is significantly less than the time taken by the MILP model.  The reason for that is that the number of iterations done in Lp is just 18 wher the number of iterations performed in MILP is 1407089. 

Also there are no bounded subproblems in LP but there are 347044 number of sub problems created in MILP which takes more time to be solved.


In [ ]:
diff = result.solver.time - result2.solver.time

print('The difference in the time ( time taken by MILP - time taken by LP) is ', diff)

The difference in the time ( time taken by MILP - time taken by LP) is  69.09486436843872
